In [660]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

In [661]:
#count schools, students and sum budget
school_count = len(school_data_complete["school_name"].unique())
student_count = len(student_data["student_name"])
budget_sum = school_data["budget"].sum()

#convert strings to numbers
school_data_complete["math_score"] = pd.to_numeric(school_data_complete["math_score"])
school_data_complete["reading_score"] = pd.to_numeric(school_data_complete["reading_score"])

#calculate averages and total passes
math_avg = school_data_complete["math_score"].mean()
reading_avg = school_data_complete["reading_score"].mean()
math_pass = school_data_complete.loc[school_data_complete["math_score"] >= 70]
reading_pass = school_data_complete.loc[school_data_complete["reading_score"] >= 70]
both_pass = school_data_complete.loc[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)]

#get percentages
math_cent = len(math_pass) / student_count
reading_cent = len(reading_pass) / student_count
passed_both_cent = len(both_pass) / student_count

district_summary_df = pd.DataFrame(
    {"Total Schools": [school_count],
     "Total Students": [student_count],
     "Total Budget": [budget_sum],
     "Average Math Score" :[math_avg],
     "Average Reading Score":[reading_avg],
     "% Passing Math":[math_cent * 100],
     "% Passing Reading":[reading_cent * 100],
     "% Overall Passing":[passed_both_cent * 100]
     }
)

district_summary_sdf = district_summary_df.style.format({"Total Students": "{:,.0f}",
                                                        "Total Budget": "${:,.2f}"})
district_summary_sdf

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,65.172326


## School Summary

In [662]:
#take columns we need from overall data and store in new frame
school_sum = school_data_complete[["school_name","type","size","budget","math_score", "reading_score", "Student ID"]]
school_sum["Per Student Budget"] = school_data_complete["budget"] / school_data_complete["size"]

#value count the number of passing students and place into data frames
mpdf = math_pass["school_name"].value_counts().to_frame().reset_index().rename(columns={"index" : "school_name", 
                                                                                 "school_name" : "math_pass"})
rpdf = reading_pass["school_name"].value_counts().to_frame().reset_index().rename(columns={"index" : "school_name", 
                                                                                 "school_name" : "reading_pass"})
bpdf = both_pass["school_name"].value_counts().to_frame().reset_index().rename(columns={"index" : "school_name", 
                                                                                 "school_name" : "both_pass"})

#left join these dataframes together
merge_df = pd.merge(school_sum, mpdf, on="school_name", how="left")
merge_df["math_pass"] = (merge_df["math_pass"] / merge_df["size"]) * 100

merge_df = pd.merge(merge_df, rpdf, on="school_name", how="left")
merge_df["reading_pass"] = (merge_df["reading_pass"] / merge_df["size"]) * 100

merge_df = pd.merge(merge_df, bpdf, on="school_name", how="left")
merge_df["both_pass"] = (merge_df["both_pass"] / merge_df["size"]) * 100

#groupby to calculate the average math and reading scores, merge this into the table
x_group = merge_df.groupby(["school_name"]).agg({"math_score": "mean", "reading_score": "mean"})
x_group = x_group.reset_index().rename(columns={"math_score":"Average Math Score","reading_score":"Average Reading Score"})
groupmerge = pd.merge(merge_df,x_group, on="school_name", how="left").drop(columns=["math_score","reading_score"])

#move the schools into the index to match example, sort alphabetically
cleaned_ssdf.index.name = None
cleaned_ssdf = cleaned_ssdf.sort_index()
duprem_cleaned_ssdf = groupmerge.drop_duplicates().set_index("school_name")

#formatting of headers
duprem_cleaned_ssdf.columns = duprem_cleaned_ssdf.columns.astype("str")
duprem_cleaned_ssdf.columns = duprem_cleaned_ssdf.columns.str.title()
duprem_cleaned_ssdf.columns = duprem_cleaned_ssdf.columns.str.replace("_", " ")
duprem_cleaned_ssdf = cleaned_ssdf.rename(columns={"Type":"School Type",
                                            "Size":"Total Students",
                                            "Budget":"Total School Budget",
                                            "Math Pass":"% Passing Math",
                                            "Reading Pass":"% Passing Reading",
                                            "Both Pass":"% Overall Passing"})

school_summary_df = cleaned_ssdf[["School Type", 
                                 "Total Students", 
                                 "Total School Budget", 
                                 "Per Student Budget", 
                                 "Average Math Score", 
                                 "Average Reading Score", 
                                 "% Passing Math", 
                                 "% Passing Reading", 
                                 "% Overall Passing"]]


school_summary_sdf = school_summary_df.style.format({"Total School Budget": "${:,.2f}",
                                                        "Per Student Budget": "${:,.2f}"})

school_summary_sdf

C:\Users\James\AppData\Local\Temp/ipykernel_4132/1567703285.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_sum["Per Student Budget"] = school_data_complete["budget"] / school_data_complete["size"]


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

In [663]:
top_5_df = school_summary_df.sort_values(by="% Overall Passing", ascending=False)
top_5_df = top_5_df.iloc[:5,:]
top_5_sdf = top_5_df.style.format({"Total School Budget": "${:,.2f}",
                                                        "Per Student Budget": "${:,.2f}"})
top_5_sdf

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [664]:
bot_5_df = school_summary_df.sort_values(by="% Overall Passing", ascending=True)
bot_5_df = bot_5_df.iloc[:5,:]
bot_5_sdf = bot_5_df.style.format({"Total School Budget": "${:,.2f}",
                                                        "Per Student Budget": "${:,.2f}"})
bot_5_sdf

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [665]:
#create a datatable for each grade with their maths average
ninthgroup = school_data_complete.loc[school_data_complete["grade"] == "9th"].groupby(["school_name"])
ninthavg = ninthgroup["math_score"].mean()
ninth_maths = ninthavg.reset_index().rename(columns={"math_score":"9th"})

tenthgroup = school_data_complete.loc[school_data_complete["grade"] == "10th"].groupby(["school_name"])
tenthavg = tenthgroup["math_score"].mean()
tenth_maths = tenthavg.reset_index().rename(columns={"math_score":"10th"})

eleventhgroup = school_data_complete.loc[school_data_complete["grade"] == "11th"].groupby(["school_name"])
eleventhavg = eleventhgroup["math_score"].mean()
eleventh_maths = eleventhavg.reset_index().rename(columns={"math_score":"11th"})

twelfthgroup = school_data_complete.loc[school_data_complete["grade"] == "12th"].groupby(["school_name"])
twelfthavg = twelfthgroup["math_score"].mean()
twelfth_maths = twelfthavg.reset_index().rename(columns={"math_score":"12th"})

#merge these into a single data table
math_score_merge = pd.merge(ninth_maths, tenth_maths, on="school_name")
math_score_merge = pd.merge(math_score_merge, eleventh_maths, on="school_name")
math_score_merge = pd.merge(math_score_merge, twelfth_maths, on="school_name")
math_score_merge = math_score_merge
math_score_merge = math_score_merge.set_index("school_name")
math_score_merge.index.name = None
math_score_merge

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [666]:
#create a datatable for each grade with their reading average
ninthgroup = school_data_complete.loc[school_data_complete["grade"] == "9th"].groupby(["school_name"])
ninthavg = ninthgroup["reading_score"].mean()
ninth_readings = ninthavg.reset_index().rename(columns={"reading_score":"9th"})

tenthgroup = school_data_complete.loc[school_data_complete["grade"] == "10th"].groupby(["school_name"])
tenthavg = tenthgroup["reading_score"].mean()
tenth_readings = tenthavg.reset_index().rename(columns={"reading_score":"10th"})

eleventhgroup = school_data_complete.loc[school_data_complete["grade"] == "11th"].groupby(["school_name"])
eleventhavg = eleventhgroup["reading_score"].mean()
eleventh_readings = eleventhavg.reset_index().rename(columns={"reading_score":"11th"})

twelfthgroup = school_data_complete.loc[school_data_complete["grade"] == "12th"].groupby(["school_name"])
twelfthavg = twelfthgroup["reading_score"].mean()
twelfth_readings = twelfthavg.reset_index().rename(columns={"reading_score":"12th"})

#merge these into a single data table
reading_score_merge = pd.merge(ninth_readings, tenth_readings, on="school_name")
reading_score_merge = pd.merge(reading_score_merge, eleventh_readings, on="school_name")
reading_score_merge = pd.merge(reading_score_merge, twelfth_readings, on="school_name")
reading_score_merge = reading_score_merge
reading_score_merge = reading_score_merge.set_index("school_name")
reading_score_merge.index.name = None
reading_score_merge

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [667]:
print(school_data_complete.columns)

Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')


In [683]:
# create bins 
bins = [0, 584.9, 629.9, 644.99, 700]
group_names = ["< $585", "$585-630", "$630-645", "$645-680"]
school_data_complete["spending_bins"] = pd.cut(school_data_complete["budget"]/school_data_complete["size"], bins, labels = group_names)

#group by spending
group_spending = school_data_complete.groupby("spending_bins")

# perform calculations
math_avg = group_spending["math_score"].mean()
reading_avg = group_spending["reading_score"].mean()
math_pass = school_data_complete[school_data_complete["math_score"] >= 70].groupby("spending_bins")['Student ID'].count()/group_spending['Student ID'].count()
reading_pass = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("spending_bins")['Student ID'].count()/group_spending['Student ID'].count()
both_pass = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)].groupby("spending_bins")['Student ID'].count()/group_spending['Student ID'].count()

#create dataframe

scores_by_spend = pd.DataFrame({
    "Average Math Score": math_avg,
    "Average Reading Score": reading_avg,
    '% Passing Math': math_pass,
    '% Passing Reading': reading_pass,
    "% Overall Passing": both_pass})

scores_by_spend



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
spending_bins,,,,,
< $585,83.363065,83.964039,0.937029,0.966866,0.906407
$585-630,79.982873,82.312643,0.791099,0.885131,0.709392
$630-645,77.821056,81.301007,0.706236,0.826002,0.588412
$645-680,77.049297,81.005604,0.662308,0.811094,0.535288
